In [ ]:
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook
from datetime import datetime

In [ ]:
def parse_and_trim(content, content_type):
    if content_type == 'HTML':
        soup = BeautifulSoup(content, 'html.parser')
    else:
        soup = BeautifulSoup(content, 'html.parser')

    for tag in soup.recursiveChildGenerator():
        try:
            tag.attrs = None
        except AttributeError:
            pass

    for linebreak in soup.find_all('br'):
        linebreak.extract()

    return soup

In [ ]:
def remove_multiple_spaces(string):
    pattern = r'\s+'
    replaced_string = re.sub(pattern, ' ', string)
    return replaced_string


def find_qrt_date(content):
    qtr_date = content.find_all(text=re.compile(
        r'for\s+(the\s+)?(fiscal\s+)?year\s+ended\s+|for\s+the\s+quarter\s+ended\s+|for\s+the\s+quarterly\s+period\s+ended\s+', re.IGNORECASE))
    qtr_match = re.search(
        r'([A-Za-z]+)\s+(\d{1,2}),\s+(\d{4})', qtr_date[0].replace('\n', ''))
    if qtr_match is None:
        qtr_match = qtr_match = re.search(
            r'([A-Za-z]+) (\d{1,2}), (\d{4})', qtr_date[1])
    return remove_multiple_spaces(str(qtr_match.group()))

In [ ]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
filing_links = pd.read_excel(
    "/Users/fuadhassan/Desktop/BDC_RA/ARCC/ARCC__sec_filing_links.xlsx")
filing_links.head()

In [ ]:
# drops all the amendment filing
filing_links = filing_links.drop(filing_links[filing_links['Form description'].str.contains(
    'amendment', case=False)].index).reset_index(drop=True)
filing_links.head()

In [ ]:
date_columns = ['Filing date', 'Reporting date']
for col in date_columns:
    filing_links[col] = pd.to_datetime(filing_links[col], format='%Y-%m-%d')
for col in date_columns:
    filing_links[col] = filing_links[col].dt.strftime("%B %d, %Y")

In [ ]:
filing_links.head()

In [ ]:
filing_links.info()

In [ ]:
# url = filing_links.iloc[1]['Filings URL']
# date = filing_links.iloc[1]['Reporting date']
# url, date

In [ ]:
# response = requests.get(url, headers=headers)
# content = parse_and_trim(response.content, 'HTML')

In [ ]:
# real one on git hub


# def extract_tables(soup_content, qtr_date):
#     master_table = None
#     all_tags = soup_content.find_all(True)
#     print(type(all_tags))
#     count = 0
#     for tag in soup_content.find_all(text=re.compile('^.*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS.*$')):
#         print('yes')
#         date_str = re.search(r'([A-Za-z]+) (\d{1,2}), (\d{4})', tag)
#         print(date_str)
#         if date_str is None:
#             next_line = tag.find_next(text=re.compile(
#                 r'([A-Za-z]+) (\d{1,2}), (\d{4})')).text
#             date_str = re.search(r'([A-Za-z]+) (\d{1,2}), (\d{4})', next_line)
#         if date_str is None:
#             next_line = tag.next.next.next.next.next.next.text
#             date_str = re.search(r'([A-Za-z]+) (\d{1,2}), (\d{4})', next_line)
#         if date_str is not None:
#             date_str = str(date_str.group())
#             date_str = unicodedata.normalize('NFKD', date_str)
#             print(date_str)
#             if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
#                 count += 1
#                 print('Table found: ')
#                 print('Table #', count)
#                 html_table = tag.find_next('table')
#                 if master_table is None:
#                     master_table = pd.read_html(
#                         html_table.prettify(), skiprows=0, flavor='bs4')[0]
#                     master_table = master_table.applymap(lambda x: unicodedata.normalize(
#                         'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
#                     master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
#                                                                                               regex=True)
#                     master_table = master_table.dropna(how='all', axis=0)
#                 else:
#                     new_table = pd.read_html(
#                         html_table.prettify(), skiprows=0, flavor='bs4')[0]
#                     new_table = new_table.applymap(lambda x: unicodedata.normalize(
#                         'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
#                     new_table = new_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
#                                                                                         regex=True)
#                     new_table = new_table.dropna(how='all', axis=0)
#                     # print('head')
#                     # print(new_table.head()) # text
#                     master_table = master_table.append(
#                         new_table.dropna(how='all', axis=0).reset_index(
#                             drop=True).drop(index=0),
#                         ignore_index=True)

#     master_table = master_table.applymap(
#         lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
#     master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
#         r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
#     return master_table

In [ ]:
def extract_tables(soup_content, qtr_date):
    master_table = None
    all_tags = soup_content.find_all(True)
    count = 0
    for tag in soup_content.find_all(text=re.compile('^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')):
        next_line_text = tag.next.text.strip()
        regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
        date_str = re.search(regex_pattern, next_line_text).group(1)

        if date_str is None:
            next_line = tag.find_next(text=re.compile(regex_pattern)).text
            date_str = re.search(regex_pattern, next_line)

        if date_str is None:
            next_line = tag.next.text.strip()
            date_str = re.search(
                regex_pattern, next_line).group(1)

        if date_str is not None:
            date_str = str(date_str)
            date_str = unicodedata.normalize('NFKD', date_str)

            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                count += 1
                print('Table #', count)
                html_table = tag.find_next('table')

                new_table = pd.read_html(
                    html_table.prettify(), skiprows=0, flavor='bs4')[0]
                new_table = new_table.applymap(lambda x: unicodedata.normalize(
                    'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                new_table = new_table.replace(
                    r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                new_table = new_table.dropna(how='all', axis=0)

                if master_table is None:
                    master_table = new_table
                else:
                    master_table = pd.concat(
                        [master_table, new_table], ignore_index=True)

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    return master_table

In [ ]:
def process_table(soi_table_df, append_str):
    soi_table_df = soi_table_df.replace(r'^\s*\$\s*$', np.nan, regex=True)
    soi_table_df = soi_table_df.dropna(how='all', axis=1)
    soi_table_df = soi_table_df.dropna(
        how='all', axis=0).reset_index(drop=True)
    # print('1: ' + str(soi_table_df.shape))

    # Separate header and data
    soi_table_header = soi_table_df.iloc[0].dropna(how='any')
    print('header: ')
    # print(soi_table_header)
    soi_table_data_df = soi_table_df.rename(
        columns=soi_table_df.iloc[0]).drop(soi_table_df.index[0])
    # print('2: ' + str(soi_table_data_df.shape))

    # Drop rows with only two non-null values
    # soi_table_data_df = soi_table_data_df.dropna(thresh=3)
    # print('4: ' + str(soi_table_data_df.shape))

    # columns_to_fill = ['Amortized Cost', 'Fair Value']
    # for col in columns_to_fill:
    #     col_index = soi_table_data_df.columns.get_loc(col)
    #     next_col_index = col_index + 1
    #     for i in range(len(soi_table_data_df)):
    #         current_value = soi_table_data_df.iat[i, col_index]

    #         if pd.isna(current_value) and next_col_index < len(soi_table_data_df.columns):
    #             next_valid_index = next((j for j, v in enumerate(
    #                 soi_table_data_df.iloc[i, next_col_index:], start=next_col_index) if pd.notna(v)), None)

    #             if next_valid_index is not None:
    #                 next_value = soi_table_data_df.iat[i, next_valid_index]
    #                 soi_table_data_df.iat[i, col_index] = next_value
    #                 soi_table_data_df.iat[i, next_valid_index] = pd.NA

    columns_to_fill = ['Amortized Cost', 'Fair  Value']

    for col in columns_to_fill:
        col_name_no_spaces = col.strip()
        if col_name_no_spaces in soi_table_data_df.columns.str.strip():
            col_index = soi_table_data_df.columns.get_loc(col_name_no_spaces)
            next_col_index = col_index + 1

            for i in range(len(soi_table_data_df)):
                current_value = soi_table_data_df.iat[i, col_index]

                if pd.isna(current_value) and next_col_index < len(soi_table_data_df.columns):
                    next_valid_index = next((j for j, v in enumerate(
                        soi_table_data_df.iloc[i, next_col_index:], start=next_col_index) if pd.notna(v)), None)

                    if next_valid_index is not None:
                        next_value = soi_table_data_df.iat[i, next_valid_index]
                        soi_table_data_df.iat[i, col_index] = next_value
                        soi_table_data_df.iat[i, next_valid_index] = pd.NA

    # Drop rows labeled as subtotals
    # subtotal_rows = soi_table_data_df[soi_table_data_df['Company (1)'].str.replace(' ', '').str.contains(
    #     'subtotal', case=False, na=False)]
    # soi_table_data_df = soi_table_data_df[~soi_table_data_df.index.isin(
    #     subtotal_rows.index)]
    # print('3: ' + str(soi_table_data_df.shape))

    # Drop rows based on regex pattern (e.g., 'subtotal' or 'total')
    pattern = r'^([Ss]ubtotal)|([Tt]otal)'
    mask = soi_table_data_df.apply(lambda row: row.astype(
        str).str.contains(pattern, case=False, na=False)).any(axis=1)
    soi_table_data_df = soi_table_data_df[~mask]
    # print('4: ' + str(soi_table_data_df.shape))

    # Drop rows with all missing values
    soi_table_df = soi_table_df.dropna(how='all')
    # print('5: ' + str(soi_table_data_df.shape))

    soi_table_data_df.to_excel('test.xlsx')

    # # Drop columns with all missing values
    soi_table_data_df = soi_table_data_df.dropna(how='all', axis=1)
    # print('6: ' + str(soi_table_data_df.shape))

    # Forward fill the first two columns
    col_indices = [0, 1]
    soi_table_data_df.iloc[:, col_indices] = soi_table_data_df.iloc[:, col_indices].fillna(
        method='ffill')
    # print('7: ' + str(soi_table_data_df.shape))

    # soi_table_data_df = soi_table_data_df.fillna(0)
    # soi_table_data_df = soi_table_data_df.dropna(how='all', axis=1)

    cols_to_convert = ['Principal', 'Amortized Cost', 'Fair Value']
    for col_name in cols_to_convert:
        if col_name in soi_table_data_df.columns:
            # print(soi_table_data_df.dtypes[col_name])
            soi_table_data_df[col_name] = pd.to_numeric(
                soi_table_data_df[col_name], errors='coerce').fillna(0)

    # columns_to_drop = []
    # for column in soi_table_data_df.columns:
    #     # Check for NaN values in the column
    #     # Use .item() to get a single boolean value
    #     if soi_table_data_df[column].isna().any().item():
    #         columns_to_drop.append(column)

    # soi_table_data_df.drop(columns=columns_to_drop, inplace=True)

    soi_table_data_df = soi_table_data_df.replace('-', 0, regex=False)
    print('8: ' + str(soi_table_data_df.shape))

    return soi_table_data_df

In [ ]:
count = 0
path = '/Users/fuadhassan/Desktop/BDC_RA/ARCC/ARCC_Investment.xlsx'
writer = pd.ExcelWriter(path, engine='openpyxl')

for qtr_date, html_link in zip(filing_links['Reporting date'], filing_links['Filings URL']):
    print('start')
    response = requests.get(html_link, headers=headers)
    content = parse_and_trim(response.content, 'HTML')
    print('content DONE')
    master_table = extract_tables(content, qtr_date)
    print(count, "master_table DONE")
    processed_table_ = process_table(
        master_table, qtr_date.replace(' ', '').replace(',', ''))
    # print(processed_table_)
    processed_table_.to_excel(
        writer, sheet_name=qtr_date.replace(' ', '').replace(',', ''), index=False)
    print(count, "processed_table_ DONE")
    count += 1
    writer.save()
    print("-------------------------------------------------------------------------------------------------------------------")
# writer.save()
writer.close()

In [ ]:
url = filing_links.iloc[27]['Filings URL']
date = filing_links.iloc[27]['Reporting date']
url, date
# response = requests.get(url, headers=headers)
# content = parse_and_trim(response.content, 'HTML')
# master_table = extract_tables(content, date)
# process_table_ = process_table(master_table, "")
# process_table_.to_excel("ex.xlsx")
# process_table_.to_csv('ex.csv')
# process_table_